##
## The goal of this Jupyter notebook is to Finetune The parameters, in order to run Co-SLAM making sure that we chose the best possible parameter for Each scene.
##

In [13]:
from datasets.dataset import ReplicaDataset
import config
import open3d as o3d
import numpy as np
import os
from tqdm import tqdm
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

def reconstruct(path_to_bound, fx, fy, cx, cy, png_depth_scale, depth_trunc, near, far):
    cfg = config.load_config(path_to_bound)
    dataset = ReplicaDataset(cfg=cfg, basedir=cfg['data']['datadir'])
    
    H, W = dataset.H, dataset.W
    K = o3d.camera.PinholeCameraIntrinsic(W, H, fx, fy, cx, cy)
    
    voxel_length = 0.005
    volume = o3d.pipelines.integration.ScalableTSDFVolume(voxel_length=voxel_length, sdf_trunc=0.04,
                                                color_type=o3d.pipelines.integration.TSDFVolumeColorType.RGB8)
    for i, frame in tqdm(enumerate(dataset)):
        if i % 2 != 0:
            continue
        rgb, depth = frame["rgb"].cpu().numpy(), frame["depth"].cpu().numpy()
        c2w = frame["c2w"].cpu().numpy()
        rgb = rgb * 255
        rgb = rgb.astype(np.uint8)
        rgb = o3d.geometry.Image(rgb)
        depth = depth / png_depth_scale  # Scale depth values
        depth = depth.astype(np.float32)
        depth = o3d.geometry.Image(depth)
        rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(rgb, depth, depth_scale=1.0,
                                                                    depth_trunc=depth_trunc,
                                                                    convert_rgb_to_intensity=False)
        #c2w[:3, 1] *= -1
        #c2w[:3, 2] *= -1
        #w2c = np.linalg.inv(c2w)

        volume.integrate(rgbd, K, c2w)
        
    print("Extracting a triangle mesh from the volume...")
    mesh = volume.extract_triangle_mesh()
    mesh.compute_vertex_normals()
    
    o3d.visualization.draw_geometries([mesh], window_name="3D Reconstruction", width=800, height=800)

def create_sliders(path_to_yaml, set_value_fx, set_value_fy, set_value_cx, set_value_cy, set_value_depthscale, set_value_depthtrunc\
                  , set_value_near, set_value_far):
    fx_slider = widgets.FloatSlider(min=500, max=2000, step=1, value=set_value_fx, description='fx')
    fy_slider = widgets.FloatSlider(min=500, max=2000, step=1, value=set_value_fy, description='fy') 
    cx_slider = widgets.FloatSlider(min=0, max=800, step=1, value=set_value_cx, description='cx')
    cy_slider = widgets.FloatSlider(min=0, max=800, step=1, value=set_value_cy, description='cy')
    depth_scale_slider = widgets.FloatSlider(min=0.1, max=10000, step=0.1, value=set_value_depthscale, description='Depth Scale')
    depth_trunc_slider = widgets.FloatSlider(min=1, max=100, step=1, value=set_value_depthtrunc, description='Depth Trunc')
    near_slider = widgets.FloatSlider(min=0.1, max=10, step=0.1, value=set_value_near, description='Near')  
    far_slider = widgets.FloatSlider(min=1, max=100, step=1, value=set_value_far, description='Far')

    interact_manual(reconstruct, 
                    path_to_bound=fixed(path_to_yaml),
                    fx=fx_slider, fy=fy_slider, cx=cx_slider, cy=cy_slider,
                    png_depth_scale=depth_scale_slider, depth_trunc=depth_trunc_slider,
                    near=near_slider, far=far_slider)

In [14]:
create_sliders(path_to_yaml='./configs/nerf/lego.yaml', set_value_fx=1111.11111, set_value_fy=1111.11111,
               set_value_cx=400.0000, set_value_cy=400.0000, set_value_depthscale=1.4, set_value_depthtrunc=10.0,
               set_value_near=0.1, set_value_far=20)

interactive(children=(FloatSlider(value=1111.11111, description='fx', max=2000.0, min=500.0, step=1.0), FloatS…

In [35]:
create_sliders(path_to_yaml = './configs/Monkey_model/monkey.yaml', set_value_fx = 711.1111, set_value_fy = 1066.6666, \
               set_value_cx = 256.0000, set_value_cy = 256.0000, set_value_depthscale = 10000.5, set_value_depthtrunc = 100.\
                  , set_value_near = 0, set_value_far = 5)

interactive(children=(FloatSlider(value=711.1111, description='fx', max=2000.0, min=500.0, step=1.0), FloatSli…

In [7]:
create_sliders(path_to_yaml='./configs/nerf/chair.yaml', set_value_fx=1111.11111, set_value_fy=1111.11111,
               set_value_cx=400.0000, set_value_cy=400.0000, set_value_depthscale=1.4, set_value_depthtrunc=10.0,
               set_value_near=0.1, set_value_far=20)

interactive(children=(FloatSlider(value=1111.11111, description='fx', max=2000.0, min=500.0, step=1.0), FloatS…

## Inversing depth Images

In [6]:
import cv2
import os

# Specify the folder containing the depth images
input_folder = "./data/lego/results/"  # Current folder

# Specify the output folder for inverted depth images
output_folder = "./data/lego/results_inv/"

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Iterate over all files in the input folder
for filename in os.listdir(input_folder):
    if filename.startswith("depth") and filename.endswith(".png"):
        # Read the depth image
        image_path = os.path.join(input_folder, filename)
        depth_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        
        # Invert the depth values
        inverted_depth = cv2.bitwise_not(depth_image)
        
        # Save the inverted depth image to the output folder
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, inverted_depth)
        
print("Depth images inverted and saved to the output folder successfully!")

Depth images inverted and saved to the output folder successfully!


## Error handling traj.txt file

In [ ]:
import os

def validate_traj_file():
    

In [17]:
import os

def validate_traj_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    for line_num, line in enumerate(lines, start=1):
        values = line.strip().split()

        if len(values) != 16:
            print(f"Error: Line {line_num} does not contain 16 values.")
            return False

        if '\t' in line:
            print(f"Error: Line {line_num} uses tab separation instead of space.")
            return False

        for value in values:
            try:
                float(value)
            except ValueError:
                print(f"Error: Line {line_num} contains a non-numeric value: {value}")
                return False

            # if 'e' in value.lower():
            #     print(f"Warning: Line {line_num} contains a value in scientific notation: {value}")

    print("traj.txt file is valid.")
    return True

traj.txt file is valid.


In [ ]:
traj_file_path = './data/SyntheticData/chair/traj.txt'

if os.path.exists(traj_file_path):
    is_valid = validate_traj_file(traj_file_path)
else:
    print(f"Error: File not found at {traj_file_path}")

In [18]:
traj_file_path = './data/Monkey_rotating/traj.txt'

if os.path.exists(traj_file_path):
    is_valid = validate_traj_file(traj_file_path)
else:
    print(f"Error: File not found at {traj_file_path}")

traj.txt file is valid.


In [21]:
traj_file_path = './data/resulting_teabox_exr/traj.txt'

if os.path.exists(traj_file_path):
    is_valid = validate_traj_file(traj_file_path)
else:
    print(f"Error: File not found at {traj_file_path}")

traj.txt file is valid.


In [22]:
import os

def check_multiple_spaces(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    for line_num, line in enumerate(lines, start=1):
        values = line.strip().split()

        for i in range(len(values) - 1):
            if values[i].endswith(' ') or values[i+1].startswith(' '):
                print(f"Warning: Line {line_num} contains multiple spaces between numbers.")
                break

    print("Multiple spaces check completed.")

# Specify the paths to your trajectory files
traj_file_paths = [
    './data/resulting_teabox_exr/traj.txt',
    './data/Monkey_rotating/traj.txt'
]

for traj_file_path in traj_file_paths:
    if os.path.exists(traj_file_path):
        print(f"Checking file: {traj_file_path}")
        check_multiple_spaces(traj_file_path)
        print()
    else:
        print(f"Error: File not found at {traj_file_path}")
        print()

Checking file: ./data/resulting_teabox_exr/traj.txt
Multiple spaces check completed.

Checking file: ./data/Monkey_rotating/traj.txt
Multiple spaces check completed.



## Visualisation of the trajectory

In [39]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interactive, IntSlider, fixed
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

def read_trajectory(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    poses = []
    for i in range(0, len(lines), 4):
        matrix = np.array([list(map(float, line.split())) for line in lines[i:i+4]])
        poses.append(matrix)
    
    return np.array(poses)

def read_trajectory_(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    poses = []
    for i in range(0, len(lines), 4):
        matrix_lines = lines[i:i+4]
        if len(matrix_lines) == 4:
            matrix = []
            for line in matrix_lines:
                values = list(map(float, line.split()))
                if len(values) == 4:
                    matrix.append(values)
            if len(matrix) == 4:
                poses.append(np.array(matrix))
    
    return np.array(poses)

def plot_trajectory(poses, frame=0):
    positions = poses[:, :3, 3]
    orientations = poses[:, :3, :3]
    
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot the trajectory
    ax.plot(positions[:, 0], positions[:, 1], positions[:, 2], 'b-')
    
    # Plot start and end points
    ax.scatter(positions[0, 0], positions[0, 1], positions[0, 2], c='g', s=100, label='Start')
    ax.scatter(positions[-1, 0], positions[-1, 1], positions[-1, 2], c='r', s=100, label='End')
    
    # Plot current position and orientation
    pos = positions[frame]
    orient = orientations[frame]
    arrow_length = 0.5
    
    ax.quiver(pos[0], pos[1], pos[2], orient[0, 0], orient[1, 0], orient[2, 0], color='r', length=arrow_length, label='X')
    ax.quiver(pos[0], pos[1], pos[2], orient[0, 1], orient[1, 1], orient[2, 1], color='g', length=arrow_length, label='Y')
    ax.quiver(pos[0], pos[1], pos[2], orient[0, 2], orient[1, 2], orient[2, 2], color='b', length=arrow_length, label='Z')
    
    # Set labels and title
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title(f'Camera Trajectory (Frame {frame})')
    
    # Add legend
    ax.legend()
    
    plt.tight_layout()
    plt.show()

In [40]:
# Load the trajectory data
file_path = './data/SyntheticData/chair/traj.txt'
poses = read_trajectory(file_path)

# Create an interactive plot
interactive_plot = interactive(plot_trajectory, 
                               poses=fixed(poses), 
                               frame=IntSlider(min=0, max=len(poses)-1, step=1, value=0, description='Frame'))
display(interactive_plot)

interactive(children=(IntSlider(value=0, description='Frame', max=49), Output()), _dom_classes=('widget-intera…

In [11]:
import numpy as np
import plotly.graph_objects as go
from ipywidgets import interactive, IntSlider
from plotly.subplots import make_subplots

def read_trajectory(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    poses = []
    for i in range(0, len(lines), 4):
        matrix = np.array([list(map(float, line.split())) for line in lines[i:i+4]])
        poses.append(matrix)
    
    return np.array(poses)

def plot_trajectory(frame=0):
    positions = poses[:, :3, 3]
    orientations = poses[:, :3, :3]
    
    fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'scene'}]])
    
    # Plot the trajectory
    fig.add_trace(go.Scatter3d(x=positions[:, 0], y=positions[:, 1], z=positions[:, 2],
                               mode='lines', line=dict(color='blue', width=2),
                               name='Trajectory'))
    
    # Plot start and end points
    fig.add_trace(go.Scatter3d(x=[positions[0, 0]], y=[positions[0, 1]], z=[positions[0, 2]],
                               mode='markers', marker=dict(size=5, color='green'),
                               name='Start'))
    fig.add_trace(go.Scatter3d(x=[positions[-1, 0]], y=[positions[-1, 1]], z=[positions[-1, 2]],
                               mode='markers', marker=dict(size=5, color='red'),
                               name='End'))
    
    # Plot current position and orientation
    pos = positions[frame]
    orient = orientations[frame]
    arrow_length = 0.5
    
    for i, color in enumerate(['red', 'green', 'blue']):
        fig.add_trace(go.Scatter3d(x=[pos[0], pos[0] + arrow_length * orient[0, i]],
                                   y=[pos[1], pos[1] + arrow_length * orient[1, i]],
                                   z=[pos[2], pos[2] + arrow_length * orient[2, i]],
                                   mode='lines', line=dict(color=color, width=3),
                                   name=f'{"XYZ"[i]}-axis'))
    
    # Set layout
    fig.update_layout(scene=dict(
                        xaxis_title='X',
                        yaxis_title='Y',
                        zaxis_title='Z',
                        aspectmode='data'),
                      title=f'Camera Trajectory (Frame {frame})',
                      height=800,
                      width=800)
    
    fig.show()

In [12]:
# Load the trajectory data
file_path = './data/SyntheticData/chair/traj.txt'
poses = read_trajectory(file_path)

# Create an interactive plot
interactive_plot = interactive(plot_trajectory, 
                               frame=IntSlider(min=0, max=len(poses)-1, step=1, value=0, description='Frame'))
display(interactive_plot)

interactive(children=(IntSlider(value=0, description='Frame', max=49), Output()), _dom_classes=('widget-intera…

## Try out the chosen Parameters.

In [10]:
from datasets.dataset import ReplicaDataset
import config
import open3d as o3d
import numpy as np
import os
from tqdm import tqdm

path_to_bound = './configs/nerf/lego.yaml'
cfg = config.load_config(path_to_bound)

dataset = ReplicaDataset(cfg=cfg, basedir=cfg['data']['datadir'])


H, W = dataset.H, dataset.W
fx, fy, cx, cy = dataset.fx, dataset.fy, dataset.cx, dataset.cy
K = o3d.camera.PinholeCameraIntrinsic(W, H, fx, fy, cx, cy)
voxel_length = 0.005
volume = o3d.pipelines.integration.ScalableTSDFVolume(voxel_length=voxel_length, sdf_trunc=0.04,
                                            color_type=o3d.pipelines.integration.TSDFVolumeColorType.RGB8)

len(dataset)

0

In [ ]:
for i, frame in tqdm(enumerate(dataset)):
    if i % 2 != 0:
         continue
    rgb, depth = frame["rgb"].cpu().numpy(), frame["depth"].cpu().numpy()
    c2w = frame["c2w"].cpu().numpy()
    rgb = rgb * 255
    rgb = rgb.astype(np.uint8)
    rgb = o3d.geometry.Image(rgb)
    depth = depth.astype(np.float32)
    depth = o3d.geometry.Image(depth)
    rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(rgb, depth, depth_scale=1.4,
                                                                depth_trunc=20.0,
                                                                convert_rgb_to_intensity=False)
    #c2w[:3, 1] *= -1
    #c2w[:3, 2] *= -1
    c2w[:3, 3] *= -1
    w2c = np.linalg.inv(c2w)

    
    # requires w2c
    volume.integrate(rgbd, K, w2c)

print("Extract a triangle mesh from the volume and visualize it.")
mesh = volume.extract_triangle_mesh()
mesh.compute_vertex_normals()


o3d.visualization.draw_geometries([mesh], window_name="3D Reconstruction", width=800, height=800)